In [1]:
import os
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split

from models.simple_cnn import SimpleCNN
from helpers.helpers import SequenceDataset

from torch.utils.data import DataLoader
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

%load_ext autoreload
%autoreload 2


In [2]:
df = pd.read_csv(f'{os.getcwd()}/data/mean_tpm.csv', index_col=0)

In [3]:
df

,species,upstream200,stress_condition,mean_tpm
0,Achromobacter xylosoxidans SOLR10,AAAAAGCAGTATCCTTGACCGACTTTGGTATGCAATGGGACTCACA...,achx_as,0.000000
1,Achromobacter xylosoxidans SOLR10,AAAAAGCAGTATCCTTGACCGACTTTGGTATGCAATGGGACTCACA...,achx_bs,0.000000
2,Achromobacter xylosoxidans SOLR10,AAAAAGCAGTATCCTTGACCGACTTTGGTATGCAATGGGACTCACA...,achx_ctrl,0.000000
3,Achromobacter xylosoxidans SOLR10,AAAAAGCAGTATCCTTGACCGACTTTGGTATGCAATGGGACTCACA...,achx_li,0.000000
4,Achromobacter xylosoxidans SOLR10,AAAAAGCAGTATCCTTGACCGACTTTGGTATGCAATGGGACTCACA...,achx_mig,0.000000
...,...,...,...,...
1122833,Vibrio cholerae O1 biovar El Tor str. N16961,TTTTTTTGAACTATAATCAACTCGTTGACTATGGGTTGGCCGGGTA...,vibrio_oss,8.015860
1122834,Vibrio cholerae O1 biovar El Tor str. N16961,TTTTTTTGAACTATAATCAACTCGTTGACTATGGGTTGGCCGGGTA...,vibrio_oxs,3.141768
1122835,Vibrio cholerae O1 biovar El Tor str. N16961,TTTTTTTGAACTATAATCAACTCGTTGACTATGGGTTGGCCGGGTA...,vibrio_sp,93.742917
1122836,Vibrio cholerae O1 biovar El Tor str. N16961,TTTTTTTGAACTATAATCAACTCGTTGACTATGGGTTGGCCGGGTA...,vibrio_tm,11.422017


In [4]:
# One-hot encode species and stress_condition
enc_species = OneHotEncoder()
encoded_species = enc_species.fit_transform(df[['species']]).toarray()

enc_stress = OneHotEncoder()
encoded_stress_condition = enc_stress.fit_transform(df[['stress_condition']]).toarray()

# Function to one-hot encode DNA sequence
def one_hot_encode_sequence(sequence, max_length=200):
    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    one_hot_encoded = np.zeros((max_length, 4))
    for i, nucleotide in enumerate(sequence[:max_length]):
        if nucleotide in mapping:
            one_hot_encoded[i, mapping[nucleotide]] = 1
    return one_hot_encoded

# Apply one-hot encoding to the DNA sequences
max_length = df['upstream200'].apply(len).max()
sequences = df['upstream200'].apply(lambda seq: one_hot_encode_sequence(seq, max_length=max_length))
sequences = np.array(sequences.tolist())

# Convert to PyTorch tensors
sequence_tensor = torch.tensor(sequences, dtype=torch.float32).permute(0, 2, 1)
species_tensor = torch.tensor(encoded_species, dtype=torch.float32)
stress_tensor = torch.tensor(encoded_stress_condition, dtype=torch.float32)
y_tensor = torch.tensor(df['mean_tpm'].values, dtype=torch.float32).unsqueeze(1)

# Create dataset
dataset = TensorDataset(sequence_tensor, species_tensor, stress_tensor, y_tensor)

# Split dataset into train, validation, and test sets
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

batch_size = 1024 * 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
class DNA_CNN(nn.Module):
    def __init__(self, num_species, num_stress_conditions, max_length=203):
        super(DNA_CNN, self).__init__()
        self.max_length = max_length
        self.conv1 = nn.Conv1d(in_channels=4, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(16 * (max_length // 2), 8)
        self.fc_species = nn.Linear(num_species, 8)
        self.fc_stress = nn.Linear(num_stress_conditions, 8)
        self.fc2 = nn.Linear(8 * 3, 1)  # 8 from CNN + 8 from species + 8 from stress

    def forward(self, x, species, stress):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 16 * (self.max_length // 2))
        x = torch.relu(self.fc1(x))

        species = torch.relu(self.fc_species(species))
        stress = torch.relu(self.fc_stress(stress))

        x = torch.cat((x, species, stress), dim=1)
        x = self.fc2(x)
        return x

In [6]:
num_species = encoded_species.shape[1]
num_stress_conditions = encoded_stress_condition.shape[1]

model = DNA_CNN(num_species, num_stress_conditions, max_length=max_length)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [7]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for step, (seqs, species, stress, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(seqs, species, stress)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        step_loss = loss.item() * seqs.size(0)
        print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{step + 1}/{len(train_loader)}], Loss: {step_loss:.4f}')
        train_loss += step_loss
    
    train_loss /= len(train_loader.dataset)

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for seqs, species, stress, targets in val_loader:
            outputs = model(seqs, species, stress)
            loss = criterion(outputs, targets)
            val_loss += loss.item() * seqs.size(0)

    val_loss /= len(val_loader.dataset)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')


Loss: 2815315.5000
Loss: 5451560.5000
Loss: 3688607.5000
Loss: 1689226.0133
Epoch [1/10], Train Loss: 3981963.9203, Validation Loss: 6803565.0000
Loss: 3339448.2500
Loss: 6168400.5000
Loss: 2686127.0000
Loss: 1413841.2995
Epoch [2/10], Train Loss: 3971197.5159, Validation Loss: 6783307.0000
Loss: 2104044.5000
Loss: 3556883.0000
Loss: 6587879.0000
Loss: 1277480.5023
Epoch [3/10], Train Loss: 3947404.4329, Validation Loss: 6747629.0000
Loss: 3706991.5000
Loss: 2452464.2500
Loss: 5750011.5000
Loss: 1495086.5441
Epoch [4/10], Train Loss: 3911878.7779, Validation Loss: 6710342.5000
Loss: 3542763.0000
Loss: 2415463.5000
Loss: 6302506.0000
Loss: 1067970.0488
Epoch [5/10], Train Loss: 3889742.9514, Validation Loss: 6712812.0000
Loss: 5565063.0000
Loss: 2833673.2500
Loss: 3039934.0000
Loss: 1922309.9098
Epoch [6/10], Train Loss: 3899162.5937, Validation Loss: 6707828.5000
Loss: 6015120.0000
Loss: 3440979.2500
Loss: 3046863.5000
Loss: 822847.4003
Epoch [7/10], Train Loss: 3888898.8567, Validatio

In [8]:
# Evaluation on test set
model.eval()
test_loss = 0.0
with torch.no_grad():
    for seqs, species, stress, targets in test_loader:
        outputs = model(seqs, species, stress)
        loss = criterion(outputs, targets)
        test_loss += loss.item() * seqs.size(0)

test_loss /= len(test_loader.dataset)
print(f'Test Loss: {test_loss:.4f}')


Test Loss: 2533004.2500


In [9]:
outputs

tensor([[307.8503],
        [333.7293],
        [305.3744],
        ...,
        [325.6265],
        [304.1159],
        [341.5404]])